In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from numpy import random
import nltk
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


from sklearn.naive_bayes import MultinomialNB
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix, classification_report
import matplotlib.pyplot as plt





In [3]:
filename = "c:/users/chung/desktop/springboard_stuff/r-depression/r-dep data cleaned.csv"
df_1 = pd.read_csv(filename)
filename_2 = "c:/users/chung/desktop/springboard_stuff/r-ptsd/r-ptsd data cleaned.csv"
df_2 = pd.read_csv(filename_2)
filename_3 = "c:/users/chung/desktop/springboard_stuff/r-cptsd/r-cptsd data cleaned.csv"
df_3 = pd.read_csv(filename_3)
filename_4 = "c:/users/chung/desktop/springboard_stuff/r-bpd/r-bpd data cleaned.csv"
df_4 = pd.read_csv(filename_4)
filename_5 = "c:/users/chung/desktop/springboard_stuff/r-bipolar/r-bipolar data cleaned.csv"
df_5 = pd.read_csv(filename_5)
filename_6 = "c:/users/chung/desktop/springboard_stuff/r-dissociation/r-diss data cleaned.csv"
df_6 = pd.read_csv(filename_6)

filename_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/depression/dep_forum_clean.csv"
df_1_f = pd.read_csv(filename_f)
filename_2_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/ptsd/ptsd_forum_clean.csv"
df_2_f = pd.read_csv(filename_2_f)
filename_3_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/cptsd/cptsd_forum_clean.csv"
df_3_f = pd.read_csv(filename_3_f)
filename_4_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/bpd/bpd_forum_clean.csv"
df_4_f = pd.read_csv(filename_4_f)
filename_5_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/bipolar/bipolar_forum_clean.csv"
df_5_f = pd.read_csv(filename_5_f)
filename_6_f = "c:/users/chung/desktop/springboard_stuff/trials/model_3/dissociation/diss_forum_clean.csv"
df_6_f = pd.read_csv(filename_6_f)

A = df_1.append(df_1_f)
B = df_2.append(df_2_f)
C = df_3.append(df_3_f)
D = df_4.append(df_4_f)
E = df_5.append(df_5_f)
F = df_6.append(df_6_f)

A_ = A.dropna()
B_ = B.dropna()
C_ = C.dropna()
D_ = D.dropna()
E_ = E.dropna()
F_ = F.dropna()

master_df = A_.append([B_,C_,D_,E_,F_])

'''assigning labels and storing in variable to prepare for holdout creation'''
df_dep = master_df.loc[master_df['label'] == 0] 
df_ptsd = master_df.loc[master_df['label'] == 1]
df_cptsd = master_df.loc[master_df['label'] == 2] 
df_bpd = master_df.loc[master_df['label'] == 3]
df_bipolar = master_df.loc[master_df['label'] == 4] 
df_diss = master_df.loc[master_df['label'] == 5]


'''creating holdouts'''
df_dep_holdout = df_dep.iloc[:100]
df_ptsd_holdout = df_ptsd.iloc[:100]
df_cptsd_holdout = df_cptsd.iloc[:100]
df_bpd_holdout = df_bpd.iloc[:100]
df_bipolar_holdout = df_bipolar.iloc[:100]
df_diss_holdout = df_diss.iloc[:100]


'''re-defining variables with holdout documents ommitted'''
df_dep = df_dep.iloc[100:]
df_ptsd = df_ptsd.iloc[100:]
df_cptsd = df_cptsd.iloc[100:]
df_bpd = df_bpd.iloc[100:]
df_bipolar = df_bipolar.iloc[100:]
df_diss = df_diss.iloc[100:]


'''concat the pieces back together'''
dfnew = pd.concat([df_dep, df_ptsd, df_cptsd, df_bpd, df_bipolar, df_diss])
df_holdout = pd.concat([df_dep_holdout, df_ptsd_holdout, df_cptsd_holdout, df_bpd_holdout, df_bipolar_holdout, df_diss_holdout])

dfnew.head()


,text,label
100,"['struggling', 'sad', 'really', 'dark', 'thoug...",0
101,"['something', 'broke', 'inside', 'never', 'gon...",0
102,"['get', 'anything', 'done', 'motivation', 'wis...",0
103,"['go', 'college', 'make', 'parent', 'happy', '...",0
104,"['maybe', 'genetics', 'maybe', 'childhood', 't...",0


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
dfnew['text'] = dfnew['text'].apply(clean_text)
df_holdout['text'] = df_holdout['text'].apply(clean_text)

dfnew.to_csv('master_rf.csv', index=False)
df_holdout.to_csv('master_rf_holdout.csv', index=False)

print(dfnew.head())
print(df_holdout.head())


X = dfnew['text'].astype(str).dropna()
y = dfnew['label']

vectorizer = TfidfVectorizer(
    stop_words='english', 
    min_df = 5,
    ngram_range = (1,2),
    use_idf = True,
    max_df = 0.5, 
    smooth_idf=True)

X = vectorizer.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2, 
    shuffle=True,
    random_state=42
)

                                                  text  label
100  struggling sad really dark thought almost 2yea...      0
101  something broke inside never gonna alright dre...      0
102  get anything done motivation wish world would ...      0
103  go college make parent happy want acknowledge ...      0
104  maybe genetics maybe childhood trauma maybe lo...      0
                                                text  label
0  know 16 people eye seems like easy get sorta p...      0
1  every moment sit pain sadness another moment l...      0
2  ever downplay anyone struggle especially come ...      0
3  struggling bad get bed sleep spend time watchi...      0
4  even know looking posting comment whatever lik...      0


In [4]:
model_dict={
    'MNB': MultinomialNB(),
    'SGD': SGDClassifier(),
    'LOG': LogisticRegression(),
    'RF': RandomForestClassifier(),
    'ADA': AdaBoostClassifier(),
    'GNB': GaussianNB(),
    'DT': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'DUM': DummyClassifier(),
}

def evaluate(model_dict, X_train, X_test, y_train, y_test):
    
    for k, v in model_dict.items():
        model = v
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

       
        print(k)
        print(classification_report(y_test, y_pred))
        
evaluate(model_dict, X_train, X_test, y_train, y_test)

MNB
              precision    recall  f1-score   support

           0       0.54      0.80      0.64      1052
           1       0.84      0.46      0.59       778
           2       0.78      0.52      0.62       861
           3       0.40      0.79      0.53      1082
           4       0.80      0.37      0.51      1004
           5       0.90      0.48      0.63       774

    accuracy                           0.58      5551
   macro avg       0.71      0.57      0.59      5551
weighted avg       0.69      0.58      0.58      5551

SGD
              precision    recall  f1-score   support

           0       0.72      0.74      0.73      1052
           1       0.76      0.71      0.73       778
           2       0.72      0.72      0.72       861
           3       0.71      0.74      0.73      1082
           4       0.76      0.79      0.77      1004
           5       0.79      0.72      0.75       774

    accuracy                           0.74      5551
   macro avg   

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LOG
              precision    recall  f1-score   support

           0       0.68      0.76      0.72      1052
           1       0.79      0.63      0.70       778
           2       0.71      0.66      0.69       861
           3       0.74      0.71      0.73      1082
           4       0.65      0.80      0.72      1004
           5       0.79      0.69      0.74       774

    accuracy                           0.72      5551
   macro avg       0.73      0.71      0.71      5551
weighted avg       0.72      0.72      0.71      5551

RF
              precision    recall  f1-score   support

           0       0.60      0.81      0.69      1052
           1       0.84      0.69      0.76       778
           2       0.80      0.64      0.71       861
           3       0.66      0.76      0.70      1082
           4       0.76      0.77      0.76      1004
           5       0.89      0.61      0.73       774

    accuracy                           0.72      5551
   macro avg    

C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\chung\anaconda3\envs\TestEnv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
